In [1]:
import pandas as pd
import numpy as np
import pyterrier_deepct
import pyterrier as pt
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [2]:
from xml.etree import ElementTree
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics2020.xml')
data = tree.getroot()
titles20 = []
titles21 = []
title_ids20 = []
title_ids21 = []
nums20 = []
nums21 = []
for topic in data:
    titles20.append(topic.find('title').text.lower()[:-1])
    num20 = topic.find('number').text
    nums20.append(num20)
tree = ElementTree.parse('/workspace/src/evaluate_targers/topics.xml')
data = tree.getroot()    
for topic in data:
    titles21.append(topic.find('title').text.lower()[:-1])
    num21 = topic.find('number').text
    nums21.append(num21)
topics = pd.DataFrame()
topics['qid'] = nums20 + nums21
topics['query'] = titles20 + titles21

topics21 = pd.DataFrame()
topics21['qid'] = nums21
topics21['query'] = titles21

topics20 = pd.DataFrame()
topics20['qid'] = nums20
topics20['query'] = titles20

qrels_path=("/workspace/src/evaluate_targers/new_qrels.qrels")
qrels = pt.io.read_qrels(qrels_path)

qrels21_path=("/workspace/src/evaluate_targers/qrels.qrels")
qrels21 = pt.io.read_qrels(qrels21_path)

qrels20_path=("/workspace/src/evaluate_targers/qrels20.qrels")
qrels20 = pt.io.read_qrels(qrels20_path)

docnos = qrels.docno

In [6]:
res

,qid,docid,docno,rank,score,query
0,51,372398,S561a86d1-A8faf4cac,0,15.063193,do we need sex education in schools
1,51,221443,S428a8bea-Aba4c7b80,1,14.124321,do we need sex education in schools
2,51,293352,S3b5af2ac-A184f2f38,2,14.112382,do we need sex education in schools
3,51,205724,S51761b31-A6016babc,3,13.676210,do we need sex education in schools
4,51,26284,Sd002be94-A3d5f97ce,4,13.647748,do we need sex education in schools
...,...,...,...,...,...,...
49977,100,106519,Sb4577bb5-A74376670,995,4.661589,do we need cash
49978,100,264144,Sbbdfefc5-Afd598db0,996,4.661340,do we need cash
49979,100,364937,Sac5ff6f2-Af7049b10,997,4.660921,do we need cash
49980,100,248510,S4d281c46-Ac290c63b,998,4.660887,do we need cash


In [21]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/"
xd = False
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            print('\n' + subdir)
            path = subdir + '/'+ file
            indexref = pt.IndexRef.of(path)
            index = pt.IndexFactory.of(indexref)
            dph = pt.BatchRetrieve(index, wmodel="DPH")
        
            res= dph.transform(topics)
            res_copy = res[res['docno'].isin(docnos)].copy()
            print(f'model_unjudged : {dph}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
            print(pt.Experiment([dph], topics, qrels, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print('-'*10)
            res= dph.transform(topics20)
            res_copy = res[res['docno'].isin(docnos)].copy()
            print(f'model_unjudged20 : {dph}, res: {pt.Utils.evaluate(res_copy,qrels20,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
            print(pt.Experiment([dph], topics20, qrels20, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
            print('-'*10)
            res= dph.transform(topics21)
            res_copy = res[res['docno'].isin(docnos)].copy()
            print(f'model_unjudged21 : {dph}, res: {pt.Utils.evaluate(res_copy,qrels21,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
            print(pt.Experiment([dph], topics21, qrels, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())  
#             xd = True
#     if xd:
#         break


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/DeepCT_indexes/conclusionspd_index
model_unjudged : BR(DPH), res: {'ndcg_cut_5': 0.5984115446906343, 'ndcg_cut_25': 0.5623335772388166, 'bpref': 0.5194695072611616}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &    bpref \\
\midrule
0 &  BR(DPH) &    0.241097 &     0.225164 &  0.51947 \\
\bottomrule
\end{tabular}

----------
model_unjudged20 : BR(DPH), res: {'ndcg_cut_5': 0.5846635565977863, 'ndcg_cut_25': 0.4917923023433454, 'bpref': 0.3918654351375907}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  BR(DPH) &    0.165038 &     0.141779 &  0.391865 \\
\bottomrule
\end{tabular}

----------
model_unjudged21 : BR(DPH), res: {'ndcg_cut_5': 0.6118845730216257, 'ndcg_cut_25': 0.6314640266363791, 'bpref': 0.6445214979422608}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midr

In [12]:
res= dph.transform(topics20)
res_copy = res[res['docno'].isin(docnos)].copy()
print(f'model_unjudged20 : {dph}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"])}')
print(pt.Experiment([dph], topics20, qrels20, eval_metrics=["ndcg_cut_5", "bpref", "ndcg_cut_25"]))
print('-'*10)

model_unjudged20 : BR(DPH), res: {'ndcg_cut_5': 0.2893789320534498, 'bpref': 0.1939535992095146, 'ndcg_cut_25': 0.24341235166488812}
      name  ndcg_cut_5     bpref  ndcg_cut_25
0  BR(DPH)    0.165038  0.391865     0.141779
----------


In [18]:
!ls /mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes

pd_index_combined-fasttext-100	   pd_index_essays-dependency-100
pd_index_deepct-conclusions1	   pd_index_essays-fasttext-100
pd_index_deepct-marco		   pd_index_ibm-fasttext-100
pd_index_deepct-original	   pd_index_original-100
pd_index_deepct-pools-conclusions  pd_index_webd-dependency-100
pd_index_deepct-test-no-split	   pd_index_webd-fasttext-100
pd_index_essays-dependency


In [20]:
import os
rootdir = "/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes"

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if(file == 'data.properties'):
            if subdir.endswith('100'):
                print('\n' + subdir)
                path = subdir + '/'+ file
                indexref = pt.IndexRef.of(path)
                index = pt.IndexFactory.of(indexref)
                dph = pt.BatchRetrieve(index, wmodel="DPH")

                res= dph.transform(topics)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model_unjudged : {dph}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
                print(pt.Experiment([dph], topics, qrels, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print('-'*10)
                res= dph.transform(topics20)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model_unjudged20 : {dph}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
                print(pt.Experiment([dph], topics20, qrels, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())
                print('-'*10)
                res= dph.transform(topics21)
                res_copy = res[res['docno'].isin(docnos)].copy()
                print(f'model_unjudged21 : {dph}, res: {pt.Utils.evaluate(res_copy,qrels,metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"])}')
                print(pt.Experiment([dph], topics21, qrels, eval_metrics=["ndcg_cut_5", "ndcg_cut_25", "bpref"]).to_latex())            


/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/targer_indexes/pd_index_webd-dependency-100
model_unjudged : BR(DPH), res: {'ndcg_cut_5': 0.5892970852533113, 'ndcg_cut_25': 0.4612504436752986, 'bpref': 0.3378988029567526}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  BR(DPH) &    0.312273 &      0.21993 &  0.337899 \\
\bottomrule
\end{tabular}

----------
model_unjudged20 : BR(DPH), res: {'ndcg_cut_5': 0.3012514912687121, 'ndcg_cut_25': 0.21506972608989625, 'bpref': 0.15181351997055773}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     bpref \\
\midrule
0 &  BR(DPH) &     0.23694 &     0.158273 &  0.306725 \\
\bottomrule
\end{tabular}

----------
model_unjudged21 : BR(DPH), res: {'ndcg_cut_5': 0.28804559398459917, 'ndcg_cut_25': 0.2461807175854023, 'bpref': 0.1860852829861949}
\begin{tabular}{llrrr}
\toprule
{} &     name &  ndcg\_cut\_5 &  ndcg\_cut\_25 &     